## 2.4 Automated Evaluation of the Q&A Robot's Performance

### 🚄 Preface

The new Q&A robot may encounter some issues during actual use, especially when users ask specific questions that require detailed knowledge from internal documents. For example, when a newcomer asks, "How to request leave," the robot might provide a generic response instead of consulting the company’s policy documents for precise guidance.

Just as conventional software development requires testing and validation, it is equally important to establish an **evaluation system** for your Q&A robot project. This ensures that similar issues can be quickly diagnosed and addressed. Moreover, after implementing any optimization or improvement, you should run a batch of test questions to confirm that the changes have a positive impact on the overall performance of the Q&A robot.

In this chapter, you will learn how to **automate evaluation processes** using LLMs and specialized frameworks like **Ragas**, enabling you to measure both the quality of answers and the effectiveness of retrieval.

## 🍁 Course Objectives
After completing this chapter, you will be able to:

- Understand how to automate evaluations for large language model (LLM) applications.
- Evaluate RAG chatbots using automated tools such as Ragas.
- Identify and solve problems in your Q&A bot through analysis of evaluation scores.

<!-- ## 📖 Course Outline
In this chapter, we will first understand some current issues with RAG chatbot through a specific problem. Then, we will attempt to discover the issue by implementing a simple automated test ourselves. Finally, we will learn how to use the more mature RAG application testing framework, Ragas, to assess the performance of RAG chatbot.

- 1.&nbsp;Evaluating RAG Application Performance
    - 1.1 Issues with the Q&A robot
    - 1.2 Checking RAG retrieval results to troubleshoot issues
    - 1.3 Attempting to establish an automated testing mechanism

- 2.&nbsp;Using Ragas to Evaluate Application Performance
     - 2.1 Evaluating the quality of responses from the RAG chatbot
       - 2.1.1 Quick start
       - 2.1.2 Understanding the calculation process of answer correctness
     - 2.2 Evaluating the recall effectiveness of retrieval
       - 2.2.1 Quick start
       - 2.2.2 Understanding the calculation process of context recall and context precision -->

## 1. Evaluating RAG Application Performance

### 1.1 Issues with the Q&A Bot

In the previous chapter, you completed the development of a Q&A bot and began exploring how to evaluate its performance.

In [2]:
import os
from config.load_key import load_key
load_key()
print(f"Your configured API Key is: {os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}")

Your configured API Key is: sk-4b*****


In [3]:
from chatbot import rag
rag.indexing()
query_engine = rag.create_query_engine(rag.load_index())
print('Question: Which department is Michael Johnson in?')
response = query_engine.query('Which department is Michael Johnson in?')
print('Answer: ', end='')
response.print_response_stream()

Question: Which department is Michael Johnson in?
Answer: Michael Johnson is in the IT Infrastructure department. He holds the position of System Administrator and works under the supervision of Michael Chen at the 456 Tech Hub.

As part of this exploration, you asked the question:

> **"Which department is Michael Johnson in?"**

And received the following answer from the bot:

> **"Michael Johnson is in the IT Infrastructure department Department. He serves as a System Administrator, working under Michael Chen at the 456 Tech Hub."**

The original document contain multiple individuals named Michael Johnson, none of whom are associated with the **Course Development Department**. The LLM, however, generated a confident and specific response that combined elements from different contexts — creating the illusion of accuracy without being grounded in factual data.

<a href="https://img.alicdn.com/imgextra/i1/O1CN01C6ZkQG1uGdQbJVw19_!!6000000006010-2-tps-1478-732.png" target="_blank">
<img src="https://img.alicdn.com/imgextra/i1/O1CN01C6ZkQG1uGdQbJVw19_!!6000000006010-2-tps-1478-732.png" width="800">
</a>

This highlights a critical issue: the answer was not based on a correct or unambiguous context, but rather on an aggregation or assumption made by the model — potentially leading to misleading conclusions.

Therefore, the next step is to examine the retrieval results used by the RAG system before generating the final answer, to determine whether the context provided was accurate and relevant to the question.

### 1.2. Check RAG Retrieval Results for Problem Diagnosis

To validate the reasoning behind the answer, we inspect the **context chunks** retrieved by the RAG system before generating the response.

Here is the retrieved context:

In [4]:
contexts = [node.get_content() for node in response.source_nodes]
contexts

['Employee Key Contact Information.md 2025-07-10\n3 / 6Employee\nIDName SupervisorOffice\nLocationPosition\nTitlePhone\nNumberEmail AddressKey\nResponsibilities\nEID-210Jennifer\nLeeMichael\nChen456\nTech\nHub\n#210Data Analyst(555)\n234-\n5687jennifer.l@educompany.comPerformance\nmetrics tracking\nEID-211Christopher\nLeeMichael\nChen456\nTech\nHub\n#211Security\nAnalyst(555)\n234-\n5688christopher.l@educompany.comSystem\nvulnerability\nassessments\nEID-212Olivia\nTaylorMichael\nChen456\nTech\nHub\n#212Project\nCoordinator(555)\n234-\n5689olivia.t@educompany.comTimeline\nmanagement\nEID-213Daniel\nSmithMichael\nChen456\nTech\nHub\n#213Frontend\nDeveloper(555)\n234-\n5690daniel.s@educompany.comUI\nimplementation\nEID-214Rachel KimMichael\nChen456\nTech\nHub\n#214Mobile App\nDeveloper(555)\n234-\n5691rachel.k@educompany.comCross-platform\ndevelopment\nEID-215Thomas\nNguyenMichael\nChen456\nTech\nHub\n#215Cloud\nEngineer(555)\n234-\n5692thomas.nguyen@educompany.comInfrastructure\nmanageme

From this context, we can see that Michael Johnson is indeed listed as a System Administrator in the Course Development Department, which directly supports the generated answer.

✅ Conclusion: The retrieval performed well — relevant and sufficient information was retrieved to support the correct answer.

While this example shows good retrieval performance, not all queries may yield such clear results. Therefore, we must build an automated evaluation framework to consistently measure retrieval quality and answer accuracy across multiple test cases.

### 1.3 Attempt to Establish an Automated Testing Mechanism

Although manual inspection helps understand individual cases, it becomes impractical when dealing with hundreds or thousands of questions. Hence, we aim to build an automated testing mechanism.

#### 1.3.1 Validating Answer Quality Using LLMs

Large language models can be leveraged not only to generate answers but also to evaluate them. By providing both the question and the generated answer, we can prompt the LLM to judge whether the answer is valid or invalid based on reference material.

Here is a function that checks if the answer effectively addresses the question.

In [5]:
from chatbot import llm

def test_answer(question, answer):
    prompt = ("You are a tester.\n"
        "You need to check whether the following answer effectively responds to the user's question.\n"
        "The reply can only be: Valid response or Invalid response. Do not provide any other information.\n"
        "------"
        f"The answer is {answer}"
        "------"
        f"The question is: {question}"
    )
    return llm.invoke(prompt,model_name="qwen-max")


test_answer("Which department is Michael Johnson in?", "Michael Johnson is in the IT Infrastructure department Department. He holds the position of System Administrator and works under the supervision of Michael Chen at the 456 Tech Hub.")

'Valid response'

The LLM confirms that the answer is valid and correctly addresses the question.



#### 1.3.2 Validating Context Relevance

Equally important is verifying that the retrieved context is useful for answering the question. We define another function to evaluate this.


In [7]:
def test_contexts(question, answer, contexts):
    prompt = (
        "You are a tester. Your task is to determine whether the provided reference materials directly support the given answer to the question.\n"
        "If the answer can be clearly found or derived from the reference materials, respond with: The reference information is useful.\n"
        "Otherwise, respond with: The reference information is not useful.\n"
        "Do not provide any other explanation or information.\n"
        "------\n"
        f"Question: {question}\n"
        f"Answer: {answer}\n"
        f"Reference materials: {' '.join(contexts)}\n"
        "------"
    )
    return llm.invoke(prompt, model_name="qwen-max")
test_contexts(
    "Which department is Michael Johnson in?", 
    "Michael Johnson is in the IT Infrastructure department Department. He holds the position of System Administrator and works under the supervision of Michael Chen at the 456 Tech Hub.", 
    contexts[0]+contexts[1]
    )

'The reference information is useful.'

#### 1.3.3 Summary of Evaluation Logic

| Component         | Method                           | Result                |
|------------------|----------------------------------|------------------------|
| Question          | "Which department is Michael Johnson in?" |                        |
| Generated Answer  | Evaluated using `test_answer`     | Valid response         |
| Retrieved Context | Evaluated using `test_contexts`   | Reference info is useful |

With the two methods above, you have already preliminarily set up a prototype of a LLMs testing project. However, the current implementation is still incomplete. For instance:

- Because LLMs sometimes hallucination, their answers may appear convincingly real. In such cases, the `test_answer` method cannot effectively detect this issue.
- The higher the proportion of relevant information in the retrieved references, the better (signal-to-noise ratio). However, our current testing method is relatively simple and does not take these factors into account.

You might consider using some mature testing frameworks to further improve your testing project. For example, [Ragas](https://docs.ragas.io/en/stable), which is a testing framework specifically designed to evaluate the performance of RAG chatbot.

## 2. Using Ragas to Evaluate Application Performance

Ragas provides multiple metrics that can be used to evaluate the quality of question-answering across the entire chain of an application. For example:

- Evaluation of overall response quality:
  - Answer Correctness, used to assess the accuracy of answers generated by the RAG application.
- Evaluation of the generation phase:
  - Answer Relevancy, used to assess whether the answers generated by the RAG application are relevant to the question.
  - Faithfulness, used to evaluate the factual consistency between the answers generated by the RAG application and the retrieved reference materials.
- Evaluation of the recall phase:
  - Context Precision, used to evaluate whether entries related to the correct answer in contexts are ranked high and have a high proportion (signal-to-noise ratio).
  - Context Recall, used to evaluate how many relevant reference materials are retrieved; a higher score means fewer relevant references are missed.

<a href="https://img.alicdn.com/imgextra/i4/O1CN01b2lVQp21JZCJy6Nfe_!!6000000006964-0-tps-739-420.jpg" target="_blank">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01b2lVQp21JZCJy6Nfe_!!6000000006964-0-tps-739-420.jpg" width="500">
</a>  



### 2.1 Evaluating the Response Quality of RAG Applications

#### 2.1.1 Quick Start  



### 2.1 Evaluating the Response Quality of RAG Applications

#### 2.1.1 Quick Start  


When evaluating the overall response quality of a RAG chatbot, using Ragas' Answer Correctness is an excellent metric. To calculate this metric, you need to prepare the following two types of data to evaluate the quality of the answer generated by the RAG chatbot:

1. question (The question input to the RAG chatbot)
2. ground_truth (The correct answer you already know)

To illustrate the differences in evaluation metrics for different responses, we have prepared three sets of RAG chatbot responses to the question:

**Question:**  
"Which department does Michael Johnson belong to?"

We will compare each model-generated **answer** against the known **ground truth**.

Three sample answers are provided below, each representing a different level of correctness:

- **Answer 1:** Based on the provided information, there is no mention of the department Michael Johnson belongs to. If you can provide more information about Michael Johnson, I may be able to help you find the answer.  
  ➤ This is considered an **invalid answer**, as it fails to provide the correct response even when context may have been available.

- **Answer 2:** Michael Johnson belongs to the Human Resources Department.  
  ➤ This is a **hallucinated answer**, as it provides a confident but incorrect response.

- **Answer 3:** Michael Johnson belongs to the Course Development Department.  
  ➤ This is the **correct answer**, matching the ground truth exactly.                                         |

We can then run the following code to calculate the score for response accuracy (i.e., Answer Correctness).

In [8]:
from langchain_community.llms.tongyi import Tongyi
from langchain_community.embeddings import DashScopeEmbeddings
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness

data_samples = {
    'question': [
        'Which department is Michael Johnson in?',
        'Which department is Michael Johnson in?',
        'Which department is Michael Johnson in?'
    ],
    'answer': [
        'According to the provided information, there is no mention of the department where Michael Johnson works. If you can provide more information about Michael Johnson, I may be able to help you find the answer.',
        'Michael Johnson is in the HR department',
        'Michael Johnson is in the Course Development Department'
    ],
    'ground_truth':[
        'Michael Johnson is a member of the Course Development Department',
        'Michael Johnson is a member of the Course Development Department',
        'Michael Johnson is a member of the Course Development Department'
    ]
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_correctness],
    llm=Tongyi(model_name="qwen-plus-0919"),
    embeddings=DashScopeEmbeddings(model="text-embedding-v3")
)
score.to_pandas()

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_correctness
0,Which department is Michael Johnson in?,"According to the provided information, there i...",Michael Johnson is a member of the Course Deve...,0.168191
1,Which department is Michael Johnson in?,Michael Johnson is in the HR department,Michael Johnson is a member of the Course Deve...,0.496046
2,Which department is Michael Johnson in?,Michael Johnson is in the Course Development D...,Michael Johnson is a member of the Course Deve...,0.998264


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>question</th>
      <th>answer</th>
      <th>ground_truth</th>
      <th>answer_correctness</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Which department does Michael Johnson belong to?</td>
      <td>According to the provided information, there is no mention of the department where Michael Johnson works. If you can provide more information about Michael Johnson, I may be able to help you find the answer.</td>
      <td>MMichael Johnson is a member of the Course Development Department</td>
      <td>0.168191</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Which department does Michael Johnson belong to?</td>
      <td>Michael Johnson belongs to the Human Resources Department.</td>
      <td>Michael Johnson is a member of the Course Development Department</td>
      <td>0.496046</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Which department does Michael Johnson belong to?</td>
      <td>Michael Johnson is in the Course Development Department</td>
      <td>Michael Johnson is a member of the Course Development Department</td>
      <td>0.998264
</td>
    </tr>
  </tbody>
</table>
</div>  



As you can see, Ragas's Answer Correctness metric accurately reflects the performance of the three responses, with the more factually accurate answers receiving higher scores.



#### 2.1.2 Understanding the Calculation Process of Answer Correctness

Intuitively, the scoring of Answer Correctness aligns with your expectations. The scoring process utilizes a large language models (in the code `llm=Tongyi(model_name="qwen-plus")`) and an embedding model (in the code `embeddings=DashScopeEmbeddings(model="text-embedding-v3")`), calculating the result based on the **semantic similarity** and **factual accuracy** between the answer and ground_truth.

##### Semantic Similarity
Semantic similarity is obtained by generating text vectors for the answer and ground_truth using the embedding model, then computing the similarity between the two text vectors. There are various methods to calculate vector similarity, such as cosine similarity, Euclidean distance, and Manhattan distance. Ragas uses the most common method, cosine similarity.

##### Factual Accuracy

Factual accuracy measures the differences in factual descriptions between the answer and ground_truth. For example, consider the following two descriptions:

- answer: Michael Johnson is a colleague in the Course Development Department responsible for big data direction.
- ground_truth: Michael Johnson is a colleague in the Course Development Department responsible for technical writer tasks.

There are factual differences between the answer and ground_truth (work direction), but there are also consistent aspects (work department). Such differences are difficult to quantify through simple calls to a LLMs or embedding model. Ragas generates respective lists of assertions for the answer and ground_truth using a LLMs and compares and calculates the elements within these assertion lists.

The following diagram can help you understand how Ragas measures factual accuracy:

<a href="https://img.alicdn.com/imgextra/i1/O1CN01OKrBYc21eB610hjF3_!!6000000007009-2-tps-1967-347.png" target="_blank">
<img src="https://img.alicdn.com/imgextra/i1/O1CN01OKrBYc21eB610hjF3_!!6000000007009-2-tps-1967-347.png" width="1000">
</a>

1. Generate respective lists of assertions for the answer and ground_truth using a LLMs. For example:
    - **Generate the assertion list for the answer**: Michael Johnson is a colleague in the Course Development Department responsible for big data direction. ---> ["*Michael Johnson is in the Course Development Department*", "*Michael Johnson is responsible for big data direction*"]
    - **Generate the assertion list for ground_truth**: Michael Johnson is a colleague in the Course Development Department responsible for technical writer tasks. ---> ["*Michael Johnson is in the Course Development Department*", "*Michael Johnson is responsible for technical writer tasks*"]

2. Traverse the assertion lists for the answer and ground_truth, initializing three lists: TP, FP, and FN.
    - For the assertions generated from the **answer**:
      - If the assertion matches one from the ground_truth, add it to the TP list. For example: "*Michael Johnson is in the Course Development Department*".
      - If the assertion cannot be found in the ground_truth list, add it to the FP list. For example: "*Michael Johnson is responsible for big data direction*".
    - For the assertions generated from the **ground_truth**:
      - If the assertion cannot be found in the answer list, add it to the FN list. For example: "*Michael Johnson is responsible for technical writer tasks*".
      > The judgment process in this step is entirely provided by the LLMs.

3. Count the number of elements in the TP, FP, and FN lists, and calculate the F1 score as follows:



```shell
f1 score = tp / (tp + 0.5 * (fp + fn)) if tp > 0 else 0
```

Taking the above text as an example: f1 score = 1/(1+0.5*(1+1)) = 0.5

##### Score Summary
After obtaining the scores for semantic similarity and factual accuracy, a weighted sum of the two can be calculated to obtain the final Answer Correctness score.  



```
Answer Correctness score = 0.25 * Semantic Similarity score + 0.75 * Factual Accuracy score
```

### 2.2 Evaluating the Effectiveness of Retrieval Recall

#### 2.2.1 Quick Start

The context precision and context recall metrics in Ragas can be used to evaluate the effectiveness of retrieval recall in RAG applications.

- Context precision evaluates whether the entries in the retrieved reference information (contexts) that are relevant to the correct answers are ranked higher and have a high proportion (signal-to-noise ratio), **focusing on relevance**.
- Context recall evaluates the factual consistency between contexts and ground_truth, **focusing on factual accuracy**.

In practical applications, these two metrics can be used together.

To calculate these metrics, your dataset should include the following information:

- **question**, the question input to the RAG application.
- **contexts**, the retrieved reference information.
- **ground_truth**, the correct answer you already know.

You can continue using the question "*Which department is Michael Johnson from?*" and prepare three sets of data. Run the code below to simultaneously calculate the scores for context precision and context recall.  



In [10]:
from langchain_community.llms.tongyi import Tongyi
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_recall, context_precision

data_samples = {
    'question': [
        'Which department is Michael Johnson in?',
        'Which department is Michael Johnson in?',
        'Which department is Michael Johnson in?'
    ],
    'answer': [
        'Based on the provided information, there is no mention of the department where Michael Johnson works. If you can provide more information about Michael Johnson, I may be able to help you find the answer.',
        'Michael Johnson is in the HR department',
        'Michael Johnson is in the Course Development Department'
    ],
    'ground_truth': [
        'Michael Johnson is a member of the Course Development Department',
        'Michael Johnson is a member of the Course Development Department',
        'Michael Johnson is a member of the Course Development Department'
    ],
    'contexts': [
        ['Provides administrative management and coordination support, optimizing administrative workflows.', 'Performance Management Department Robert Carter EID-701 Course Development Department'],
        ['Michael Chen, Director of the Course Development Department', 'Newton discovered the law of universal gravitation'],
        ['Newton discovered the law of universal gravitation', 'Michael Johnson, engineer in the Course Development Department, has recently been responsible for technical writer tasks.'],
    ],
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset=dataset,
    metrics=[context_recall, context_precision],
    llm=Tongyi(model_name="qwen-plus-0919"))
score.to_pandas()

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

,question,answer,ground_truth,contexts,context_recall,context_precision
0,Which department is Michael Johnson in?,"Based on the provided information, there is no...",Michael Johnson is a member of the Course Deve...,[Provides administrative management and coordi...,1.0,0.0
1,Which department is Michael Johnson in?,Michael Johnson is in the HR department,Michael Johnson is a member of the Course Deve...,"[Michael Chen, Director of the Course Developm...",1.0,0.0
2,Which department is Michael Johnson in?,Michael Johnson is in the Course Development D...,Michael Johnson is a member of the Course Deve...,[Newton discovered the law of universal gravit...,1.0,0.5


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>question</th>
      <th>answer</th>
      <th>ground_truth</th>
      <th>contexts</th>
      <th>context_recall</th>
      <th>context_precision</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Which department is Michael Johnson in?</td>
      <td>Based on the provided information, there is no mention of the department where Michael Johnson works. If you can provide more information about Michael Johnson, I may be able to help you find the answer.</td>
      <td>Michael Johnson is a member of the Course Development Department.</td>
      <td>[Provides administrative management and coordination support, optimizing administrative workflows., Performance Management Department Robert Carter EID-701 Course Development Department]</td>
      <td>1.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Which department is Michael Johnson in?</td>
      <td>Michael Johnson is in the HR department</td>
      <td>Michael Johnson is a member of the Course Development Department.</td>
      <td>[Michael Chen, Director of the Course Development Department, Newton discovered the law of universal gravitation]</td>
      <td>1.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Which department is Michael Johnson in?</td>
      <td>Michael Johnson is in the Course Development Department</td>
      <td>Michael Johnson is a member of the Course Development Department.</td>
      <td>[Newton discovered the law of universal gravitation, Michael Johnson, engineer in the Course Development Department, has recently been responsible for curriculum development]</td>
      <td>1.0</td>
      <td>0.5</td>
    </tr>
  </tbody>
</table>
</div>

From the data above, we can see that:
- The answer in the last row of data is accurate.
- The reference materials (contexts) retrieved during the process also contain the correct viewpoint, i.e., "Michael Johnson belongs to the Course Development Department." This situation is reflected in the context recall score being 1.
- However, not every piece of information in the contexts is relevant to the question and answer. For example, "Newton discovered gravity." This situation is reflected in the context precision score being 0.5.

#### 2.2.2 Understanding the calculation process of context recall and context precision

##### Context Recall

You have already learned from the previous text that context recall is a metric used to measure whether contexts are consistent with ground_truth.

In Ragas, context recall is used to describe what proportion of viewpoints in ground_truth can be supported by contexts. The calculation process is as follows:

1. A large language model (LLM) breaks down the ground_truth into n statements.

   For example, from the ground_truth "Michael Johnson is a member of the Course Development Department," a list of statements such as ["Michael Johnson belongs to the Course Development Department"] can be generated.
2. The LLM determines whether each statement can find supporting evidence in the retrieved reference materials (contexts), or whether the context can support the viewpoint of the ground_truth.

   For instance, this statement can find supporting evidence in the third row of data's contexts: "*Michael Johnson, engineer in the Course Development Department, has recently been responsible for curriculum development.*"
3. Then, the proportion of statements in the ground_truth list that can find supporting evidence in the contexts is calculated as the context_recall score.

   Here, the score is 1 = 1/1.

##### Context Precision

In Ragas, context precision not only measures what proportion of contexts are related to the ground_truth but also evaluates the ranking of contexts. The calculation process is more complex:

1. Read through the contexts sequentially, and based on the question and ground_truth, determine whether context<sub>i</sub> is relevant. If relevant, it scores 1; otherwise, it scores 0.

   For example, in the third row of data, context<sub>1</sub> ("Newton discovered gravity") is irrelevant, while context<sub>2</sub> is relevant.
2. For each context, calculate the precision score by dividing the cumulative sum of scores of the current context and all preceding contexts (numerator) by the position of the context in the sequence (denominator).

   For the third row of data, the precision score for context<sub>1</sub> is 0/1 = 0, and for context<sub>2</sub>, it is 1/2 = 0.5.
3. Sum up the precision scores of all contexts and divide by the number of relevant contexts to obtain the context_precision.

   For the third row of data, context_precision = (0 + 0.5) / 1 = 0.5.

> If you cannot fully understand the calculation process above, it doesn't matter. You only need to know that this metric evaluates the ranking of contexts. If you're interested, we encourage you to read [Ragas's source code](https://github.com/explodinggradients/ragas/blob/cc31f65d4b7c7cd6bbf686b9073a0dfaacfbcbc5/src/ragas/metrics/_context_precision.py#L250).

### 2.3 Other Recommended Metrics to Explore

Ragas also provides many other metrics, which will not be introduced one by one here. You can visit the Ragas documentation to learn more about the applicable scenarios and working principles of these metrics.

The metrics supported by Ragas can be accessed at: https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/  



## 3. How to Optimize Based on Ragas Metrics
The ultimate goal of evaluation is not to obtain scores, but to determine the direction of optimization based on these scores. You have already learned the concepts and calculation methods of three metrics: answer correctness, context recall, and context precision. When you observe that the scores of certain metrics are low, corresponding optimization measures should be formulated.

### 3.1 Context Recall
The context recall metric evaluates the performance of a RAG application during the **retrieval** phase. If this metric has a low score, you can try optimizing from the following aspects:

- **Check the Knowledge Base**

    <img src="https://wanx.alicdn.com/wanx/1937257750879544/text_to_image_lite_v2/6e4ca1055a0c467992b6b719b443a33e_0.png?x-oss-process=image/watermark,image_aW1nL3dhdGVybWFyazIwMjQxMTEyLnBuZz94LW9zcy1wcm9jZXNzPWltYWdlL3Jlc2l6ZSxtX2ZpeGVkLHdfMzAzLGhfNTI=,t_80,g_se,x_10,y_10/format,webp" width="300">

    The knowledge base is the source of a RAG application. If the content of the knowledge base is incomplete, it will lead to insufficient reference information being recalled, thereby affecting context recall. You can compare the content of the knowledge base with test samples and observe whether the content of the knowledge base can support each test sample (this process can also be assisted by LLMs. If you find that some test samples lack relevant knowledge, you need to supplement the knowledge base.
- **Replace the Embedding Model**

    <img src="https://img.alicdn.com/imgextra/i4/O1CN01MMsV3b1U2GzviZv6y_!!6000000002459-2-tps-991-320.png" width="750">

    If your knowledge base content is already very complete, consider replacing the embedding model. A good embedding model can understand the deep semantic meaning of text. If two sentences are deeply related, even if they don't appear to be related, they can still receive a high similarity score. For example, if the question is "Who is responsible for curriculum development?" and the corresponding text segment in the knowledge base is "Michael Johnson is a member of the Course Development Department," despite fewer overlapping words, an excellent embedding model can still assign a high similarity score to these two sentences, thereby recalling the text segment "Michael Johnson is a member of the Course Development Department."
- **query rewriting**

    <img src="https://img.alicdn.com/imgextra/i1/O1CN01RpktVQ1FEtg8r4QCX_!!6000000000456-2-tps-1704-1322.png" width="800">

    As a developer, it is unrealistic to impose too many requirements on how users ask questions. Therefore, you might receive vague questions like: "Course Development Department," "Leave Request," "Project Management." If such questions are directly input into a RAG application, they are unlikely to recall effective text segments. You can design a prompt template by organizing common employee questions and use LLMs to rewrite queries, improving the accuracy of recall.


### 3.2 Context Precision
Similar to context recall, the context precision metric also evaluates the performance of a RAG application during the **retrieval** phase, but it focuses more on whether related text segments have higher rankings. If this metric has a low score, you can try the optimization measures mentioned under context recall, and you can also attempt to add **reranking** during the retrieval phase to improve the ranking of related text segments.


### 3.3 Answer Correctness
The answer correctness metric evaluates the overall comprehensive performance of a RAG system. If this metric has a low score while the previous two metrics have high scores, it indicates that the RAG system performs well in the **retrieval** phase but encounters issues in the **generation** phase. You can try the methods learned in previous tutorials, such as optimizing prompts, adjusting hyperparameters (such as temperature) of LLMs generation, or replacing with a more powerful LLMs, and even fine-tuning the LLMs (which will be introduced in later tutorials) to enhance the accuracy of generated answers.

## ✅ Summary of this section

Through the study of this section, you have learned how to establish automated testing for RAG chatbots.

Automated testing is an important means of engineering optimization. With quantified automated testing, it can help you shift from **feeling** better to metrics **quantization** that the application performs better when improving your RAG chatbot. This not only helps you evaluate the question-answering quality of the RAG chatbot faster and find optimization directions, but also quantifies the optimization results you have achieved.

Of course, having automated testing does not mean that you no longer need human evaluation at all. It is recommended that in practical applications, you invite domain experts corresponding to the RAG chatbot to jointly build a test set that reflects the distribution of real-world scenario problems, and continuously update the test set.

At the same time, since LLMs cannot always achieve 100% accuracy, it is also recommended that you regularly sample and evaluate the accuracy of automated testing results during actual use, and try not to frequently change the LLMs and embedding models. For Ragas, you can improve its performance by adjusting the prompts in the default evaluation method (for example, supplementing reference examples related to your business domain) (for more details, please refer to the extended reading).  



## Further Reading

### Changing the Prompt Template in Ragas
Many of Ragas's evaluation metrics rely on large language models for implementation. Like LlamaIndex, Ragas provides default prompt templates but also supports custom prompts that you can modify to suit your needs.

We have included example prompt templates in the `ragas_prompt` folder to help you customize the prompts used by Ragas for different evaluation metrics. You can refer to the following code to integrate these updated prompts into your workflow.

> Note: Ragas includes example cases in its prompts to guide the model on how to make judgments or generate lists of statements. You can replace or modify these examples to better align with your specific use case or domain.

In [11]:
# Import prompt templates
from ragas_prompt.ragas_test_prompt import ContextRecall, ContextPrecision, AnswerCorrectness

# Customize prompt settings for each metric
context_recall.context_recall_prompt.instruction = ContextRecall.context_recall_prompt["instruction"]
context_recall.context_recall_prompt.output_format_instruction = ContextRecall.context_recall_prompt["output_format_instruction"]
context_recall.context_recall_prompt.examples = ContextRecall.context_recall_prompt["examples"]

context_precision.context_precision_prompt.instruction = ContextPrecision.context_precision_prompt["instruction"]
context_precision.context_precision_prompt.output_format_instruction = ContextPrecision.context_precision_prompt["output_format_instruction"]
context_precision.context_precision_prompt.examples = ContextPrecision.context_precision_prompt["examples"]

answer_correctness.correctness_prompt.instruction = AnswerCorrectness.correctness_prompt["instruction"]
answer_correctness.correctness_prompt.output_format_instruction = AnswerCorrectness.correctness_prompt["output_format_instruction"]
answer_correctness.correctness_prompt.examples = AnswerCorrectness.correctness_prompt["examples"]

data_samples = {
    'question': [
        'Which department is Michael Johnson in?',
        'Which department is Michael Johnson in?',
        'Which department is Michael Johnson in?'
    ],
    'answer': [
        'Based on the provided information, there is no mention of the department where Michael Johnson works. If you can provide more information about Michael Johnson, I may be able to help you find the answer.',
        'Michael Johnson is in the HR department',
        'Michael Johnson is in the Course Development Department'
    ],
    'ground_truth': [
        'Michael Johnson is a member of the Course Development Department',
        'Michael Johnson is a member of the Course Development Department',
        'Michael Johnson is a member of the Course Development Department'
    ],
    'contexts': [
        ['Provides administrative management and coordination support, optimizing administrative workflows.', 'Performance Management Department Han Shan Li Fei I902 041 Human Resources'],
        ['Li Kai, Director of the Course Development Department', 'Newton discovered the law of universal gravitation'],
        ['Newton discovered the law of universal gravitation', 'Michael Johnson, engineer in the Course Development Department, has recently been responsible for curriculum development.'],
    ],
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(
    dataset=dataset,
    metrics=[answer_correctness, context_recall, context_precision],
    llm=Tongyi(model_name="qwen-plus-0919"),
    embeddings=DashScopeEmbeddings(model="text-embedding-v3"))

score.to_pandas()

Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

,question,answer,ground_truth,contexts,answer_correctness,context_recall,context_precision
0,Which department is Michael Johnson in?,"Based on the provided information, there is no...",Michael Johnson is a member of the Course Deve...,[Provides administrative management and coordi...,0.166901,0.0,0.0
1,Which department is Michael Johnson in?,Michael Johnson is in the HR department,Michael Johnson is a member of the Course Deve...,"[Li Kai, Director of the Course Development De...",0.196046,1.0,0.0
2,Which department is Michael Johnson in?,Michael Johnson is in the Course Development D...,Michael Johnson is a member of the Course Deve...,[Newton discovered the law of universal gravit...,0.998264,1.0,0.5


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>question</th>
      <th>answer</th>
      <th>ground_truth</th>
      <th>contexts</th>
      <th>answer_correctness</th>
      <th>context_recall</th>
      <th>context_precision</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Which department is Michael Johnson in?</td>
      <td>Based on the provided information, there is no mention of the department where Michael Johnson works. If you can provide more information about Michael Johnson, I may be able to help you find the answer.</td>
      <td>Michael Johnson is a member of the Course Development Department.</td>
      <td>[Providing administrative management and coordination support, optimizing administrative workflows. , Performance Management Department Han Shan Li Fei I902 041 ...</td>
      <td>0.166901</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Which department is Michael Johnson in?</td>
      <td>Michael Johnson is in the Human Resources Department.</td>
      <td>Michael Johnson is a member of the Course Development Department.</td>
      <td>[Li Kai, Director of the Course Development Department , Newton discovered the law of universal gravitation]</td>
      <td>0.196046</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Which department is Michael Johnson in?</td>
      <td>Michael Johnson is in the Course Development Department.</td>
      <td>Michael Johnson is a member of the Course Development Department.</td>
      <td>[Newton discovered the law of universal gravitation, Michael Johnson, an engineer in the Course Development Department, has recently been responsible for curriculum development]</td>
      <td>0.998264</td>
      <td>1.0</td>
      <td>0.5</td>
    </tr>
  </tbody>
</table>
</div>  



### More Evaluation Metrics
In addition to RAG, there are many applications or tasks of LLMs or natural language processing (NLP), such as Agents, natural language to SQL, machine translation, summarization, etc. Ragas provides many metrics that can be used to evaluate these tasks.


| Evaluation Metric            | Use Case | Metric Meaning                                                                 |
|---------------------|----------|--------------------------------------------------------------------------|
| [ToolCallAccuracy](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/agents/#example)    | Agent    | Evaluates the LLM's performance in identifying and invoking tools required to complete specific tasks. This metric is obtained by comparing reference tool calls with tool calls made by the LLM, with a value range of 0-1. |
| [DataCompyScore](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/sql/)      | natural language to SQL   | Evaluates the difference between the results obtained from database queries using SQL statements generated by the LLM and the correct results. The value ranges from 0 to 1.                     |
| [LLMSQLEquivalence](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/sql/#non-execution-based-metrics)   | natural language to SQL   | Unlike the previous metric, this does not require actual database retrieval; it only evaluates the differences between SQL statements generated by the LLM and the correct SQL statements. The value ranges from 0 to 1.   |
| [BleuScore](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/traditional/#bleu-score)           | General     | Evaluates the similarity between responses and correct answers based on n-grams. Initially designed for evaluating machine translation systems, this metric does not require the use of an LLM during evaluation, and its value ranges from 0 to 1. In the [2.7 tutorial](2_7_Improve_Model_Accuracy_and_Efficiency_via_Fine_Tuning.ipynb), you will learn how to fine-tune LLMs, and BleuScore can be used to measure the benefits brought by fine-tuning.  



## 🔥 Post-class Quiz

### 🔍 Single-choice Question
<details>
<summary style="cursor: pointer; padding: 12px;  border: 1px solid #dee2e6; border-radius: 6px;">
<b>What does the Context Precision metric measure? ❓</b>

- A. Evaluation of overall response quality
- B. Evaluation of whether retrieved text segments relevant to the question are ranked higher
- C. Whether the generated answer is related to the retrieved text segments
- D. Whether the generated answer is relevant to the question

**[Click to view the answer]**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **Reference Answer: B**  
📝 **Explanation**:  
- Context Precision directly evaluates the ranking quality of retrieval results, not the relevance of the content itself or the quality of the final answer.

</div>
</details>  



## ✉️ Evaluation and Feedback
Thank you for studying the Alibaba Cloud Large Model ACP Certification course. If you think there are parts of the course that are well-written or need improvement, we look forward to your [evaluation and feedback through this questionnaire](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie).

Your criticism and encouragement are both driving forces for our progress.  

